In [49]:
from urllib.request import Request, urlopen
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import time

In [3]:
# Create reference to main webpage that will be used to seek out various car makes and models
url = 'https://www.carspecs.us/'
response = requests.get(url)

In [5]:
# The beautiful soup result of parsing the main page. Will search for links to pursue
bsMain = BeautifulSoup(response.text,"html.parser")

In [57]:
# Indices 24 to 91 are the car links
MainLinkList = bsMain.findAll('a')
MainLinkList[24]['href']

'/cars/acura'

In [66]:
# This gets the specific make domain for appending to the main link
make = MainLinkList[24]['href']

# Appending the link for car make to the homepage link. This leads to the list of years
NewUrl = url + make[1:]

# Updated URL Request
NewResponse = requests.get(NewUrl)

# Updated Beautiful Soup of years
bsYears = BeautifulSoup(NewResponse.text,'html.parser')



In [85]:
# Find all links on the page of car years
bsYears.findAll('a')

# Find the index of the respective year. Initially I'll only consider 2018 and 2019 cars
yr2019 = [i for i, j in enumerate(bsYears.findAll('a')) if '2019' in j][0]
# yr2018 = [i for i, j in enumerate(bsYears.findAll('a')) if '2018' in j][0]

In [89]:
# Get the year part of link to append to make like
year = bsYears.findAll('a')[yr2019]['href']

# New url with years
NewUrl = url + year[1:]

In [90]:
# The URL works. Now the challenge is to find the car models.
# I'm not 100% sure how to go about this since every car manufacturer has a unique
# naming criteria for their car models. If I find consistent starting indices I might
# be able to leverage that.

'https://www.carspecs.us/cars/2019/acura'